In [31]:
from numpy import cos
from numpy import exp
from numpy import sin
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy import hstack
from numpy import delete
from numpy import sqrt
from numpy import arctan2
from numpy import pi

In [32]:
def coefficientsDeriche(alpha, omega):

    c=(1-2*exp(-alpha)*cos(omega)+exp(-2*alpha))/(exp(-alpha)*sin(omega))
    k=((1-2*exp(-alpha)*cos(omega)+exp(-2*alpha))*(pow(alpha,2)+pow(omega,2)))/(2*alpha*exp(-alpha)*sin(omega)+omega-omega*exp(-2*alpha))

    c1=(k*alpha)/(pow(alpha,2)+pow(omega,2))
    c2=(k*omega)/(pow(alpha,2)+pow(omega,2))

    b1=-2*exp(-alpha)*cos(omega)
    b2=exp(-2*alpha)

    a=-c*exp(-alpha)*sin(omega)
    a0=c2
    a1=(-c2*cos(omega)+c1*sin(omega))*exp(-alpha)
    a2=a1-c2*b1
    a3=-c2*b2
    
    return c,k,c1,c2,b1,b2,a,a0,a1,a2,a3
#enddef

In [33]:
def suppnonmaxima(normeGradient, arg):
    [n,p]=normeGradient.shape
    masque = zeros([n,p])

    arg[arg<0]=arg[arg<0]+180
    arg[(((arg>=0) & (arg<22.5)) | ((arg>=157.5) & (arg<=180)))]=1
    arg[((arg>=22.5) & (arg<67.5))]=2
    arg[((arg>=67.5) & (arg<112.5))]=3
    arg[((arg>=112.5) & (arg<157.5))]=4
 
    for i in range(1,n-1,1):
        for j in range(1,p-1,1):
            if arg[i,j]==1: # [0 22.5[ ou [157.5 180] degres
                masque[i,j] = (normeGradient[i,j]>=max(normeGradient[i,j+1],normeGradient[i,j-1]))
            elif arg[i,j]==2: # [22.5 67.5[ degres
                masque[i,j] = (normeGradient[i,j]>=max(normeGradient[i-1,j+1],normeGradient[i+1,j-1]))
            elif arg[i,j]==3: # [67.5 112.5[ degres
                masque[i,j] = (normeGradient[i,j]>=max(normeGradient[i-1,j],normeGradient[i+1,j]))
            else: # arg[i,j]==4 # [112.5 157.5[ degres
                masque[i,j] = (normeGradient[i,j]>=max(normeGradient[i-1,j-1],normeGradient[i+1,j+1]))
        #endfor
    #endfor 

    normeGradient[masque==0] = 0
    
    return normeGradient
#enddef


In [34]:
def seuillageParHysteresis(res, Smin, Smax):
    pythonOblige=1
    N=3
    P=3
    [n,p]=res.shape
    
    resAvecZeros=vstack((zeros([1,p+(P-1)]), hstack((zeros([n,1]), res, zeros([n,1]))), zeros([1,p+(P-1)])))

    [nbLigne,nbColonne]=resAvecZeros.shape
    resFinal=-ones([nbLigne,nbColonne])
    
    for i in range (0,nbLigne-1,1):
        for j in range (0,nbColonne-1,1):
            
            if (resAvecZeros[i,j]<=Smin):
                resFinal[i,j]=0
            else:
                if ((resAvecZeros[i,j]>Smin) & (resAvecZeros[i,j]<Smax)):
                    fenetre=resAvecZeros[(i-1):((i+1)+pythonOblige),(j-1):((j+1)+pythonOblige)]
                    if ~fenetreContientValSupEgaleSmax(fenetre,Smax,N,P):
                        resFinal[i,j]=0                        
                    else:
                        resFinal[i,j]=255
                    #endif
                else:
                    #if (resAvecZeros[i,j]>=Smax)
                    resFinal[i,j]=255
                    #endif
                #endif
            #endif
        #endfor
    #endfor
    
    # delete(array, indicate which sub-arrays to remove, axis) 0 : horizontalement 1 : verticalement
    resFinal=delete(resFinal,0,1)
    [nbLigne,nbColonne]=resFinal.shape
    resFinal=delete(resFinal,nbColonne-1,1)
    [nbLigne,nbColonne]=resFinal.shape
    resFinal=delete(resFinal,0,0)
    [nbLigne,nbColonne]=resFinal.shape
    resFinal=delete(resFinal,nbLigne-1,0)
    
    return resFinal
#end
            
def fenetreContientValSupEgaleSmax(fenetre,Smax,N,P):
    i=0
    
    stop=False
    while ((i<N) & (stop==False)):
        j=0
        while ((j<P) & (stop==False)):
            if fenetre[i,j]>=Smax:
                stop=True
            else:
                j=j+1
            #endif
        #endwhile
        i=i+1
    #endwhile
    return stop
#endef

In [35]:
def filtreDericheRecursif(image, b1, b2, a, a0, a1, a2, a3):
    [n,p]=image.shape
    
    # Gradient
    # Y+
    Yp=zeros([n,p])

    debut_j=0
    #Yp[:,debut_j]=zeros([n,1])
    debut_j=debut_j+1
    Yp[:,debut_j]=image[:,debut_j-1]-b1*Yp[:,debut_j-1]
    debut_j=debut_j+1
    for j in range(debut_j,p,1):
        Yp[:,j]=image[:,j-1]-b1*Yp[:,j-1]-b2*Yp[:,j-2]
    #endfor

    # Y-
    Ym=zeros([n,p])

    fin_j=p-1
    #Ym[:,fin_j]=zeros([n,1])
    fin_j=fin_j-1
    Ym[:,fin_j]=image[:,fin_j+1]-b1*Ym[:,fin_j+1]
    fin_j=fin_j-1
    for j in range(fin_j,-1,-1):
        Ym[:,j]=image[:,j+1]-b1*Ym[:,j+1]-b2*Ym[:,j+2]
    #endfor

    # S
    S=a*(Yp-Ym)
    
    # Lissage
    # R+
    Rp=zeros([n,p])

    debut_i=0
    Rp[debut_i,:]=a0*S[debut_i,:]
    debut_i=debut_i+1
    Rp[debut_i,:]=a0*S[debut_i,:]+a1*S[debut_i-1,:]-b1*Rp[debut_i-1,:]
    debut_i=debut_i+1
    for i in range(debut_i,n,1):
        Rp[i,:]=a0*S[i,:]+a1*S[i-1,:]-b1*Rp[i-1,:]-b2*Rp[i-2,:]
    #endfor

    # R-
    Rm=zeros([n,p])

    fin_i=n-1
    #Rm[fin_i,:]=zeros([1,p])
    fin_i=fin_i-1
    Rm[debut_i,:]=a2*S[fin_i+1,:]-b1*Rm[fin_i+1,:]
    fin_i=fin_i-1
    for i in range(fin_i,-1,-1):
        Rm[i,:]=a2*S[i+1,:]+a3*S[i+2,:]-b1*Rm[i+1,:]-b2*Rm[i+2,:]
    #endfor

    # R
    R=Rm+Rp
    
    return R
#enddef

In [36]:
def mainCannyDeriche(imageGray,alpha,omega,Smin,Smax):
    # Les coefficients du filtre
    [c,k,c1,c2,b1,b2,a,a0,a1,a2,a3]=coefficientsDeriche(alpha,omega)
    
    #### Gradient X et Lissage
    gradientX = filtreDericheRecursif(imageGray, b1, b2, a, a0, a1, a2, a3)

    #plt.imshow(gradientX)
    #plt.title('Gradient X'), plt.xticks([]), plt.yticks([])
    #plt.show()
    
    #### Gradient Y et Lissagee
    gradientYtranspose = filtreDericheRecursif(imageGray.T, b1, b2, a, a0, a1, a2, a3);
    gradientY=gradientYtranspose.T;

    #plt.imshow(gradientY)
    #plt.title('Gradient Y'), plt.xticks([]), plt.yticks([])
    #plt.show()
    
    #### Norme du gradient (magniture)
    normeGradient = sqrt(pow(gradientX,2) + pow(gradientY,2))

    #plt.imshow(normeGradient)
    #plt.title('Norme du gradient'), plt.xticks([]), plt.yticks([])
    #plt.show()
    
    #### Direction du gradient
    directionGradient = arctan2(gradientY,gradientX); # en radian
    directionGradient = directionGradient*180/pi # conversion en degre

    #plt.imshow(directionGradient)
    #plt.title('Direction du gradient'), plt.xticks([]), plt.yticks([])
    #plt.show()
    
    #### Suppression des non-maxima locaux
    res_suppnonmax = suppnonmaxima(normeGradient, directionGradient)

    #plt.imshow(res_suppnonmax, cmap = 'gray')
    #plt.title('Après suppression des non-maxima locaux'), plt.xticks([]), plt.yticks([])
    #plt.show()
    
    #### Hysteresis thresholding (using two thresholds)
    res = seuillageParHysteresis(res_suppnonmax, Smin, Smax)

    #plt.figure(figsize = (50,10))
    #plt.imshow(res, cmap = 'gray')
    #plt.title('Après seuillage par hysteresis'), plt.xticks([]), plt.yticks([])
    #plt.show()
    
    return res
#enddef